## Import Libraries

In [1]:
import sys
sys.path.append('../')

from groof.config import * # IMPORTS ALL DIRECTORIES AND FILE PATHS

from groof.ee_indices import * 
import geemap
from groof.geo import *
from groof.ee_pipeline import *
from shapely.geometry import box
import geopandas as gpd
import ee

## Get ROI

In [2]:
bbox = gpd.read_file(BBOX)
bbox_coords = list(bbox.to_crs(4326).total_bounds)
utm_crs = bbox.estimate_utm_crs()


## Authenticate GEE

In [3]:
ee.Authenticate()
ee.Initialize(project='ee-geoai-medo')

# Get Sentinel-2 Data

In [4]:
geom = ee.Geometry.Rectangle(bbox_coords)


## Fetch Sentinel-2 Image
ee_s2.update(dict(bands = S2_BANDS))
s2_image = ee_get_s2(ee_s2, geom, start_date='2021-01-01', end_date='2025-12-30', max_cloud=15).map(mask_s2_clouds)
s2_image = s2_image.filter(ee.Filter.calendarRange(3,6, 'month')).map(lambda img: img.divide(10000))


# Median Image 
s2_median = s2_image.median().clip(geom)
s2_focal = s2_median.focalMedian(1.5, 'square', 'pixels')

il_bldg = ee.FeatureCollection('projects/sat-io/open-datasets/ORNL/USA-STRUCTURES/USA_ST_IL')
il_bldg = il_bldg.filterBounds(geom)

## Compute Spectral Indices

In [5]:
NIR = s2_median.select('B8')
RED = s2_median.select('B4')
BLUE = s2_median.select('B2')
SWIR = s2_median.select('B11')
GREEN = s2_median.select('B2')
SWIR2 = s2_median.select('B12')

## Normalized Difference Built-up Index
ndbi_median = ndbi_ee_s2(s2_median, geom)

# Built-up Index (BU)
##  NDDBI - NDVI
bu_median = bu_ee_s2(s2_median, geom)

# ENHANCED VEGETATION INDEX (EVI)
evi_median = evi_ee_s2(s2_median, geom)

# NORMALIZED DIFFERENCE WATER INDEX
## NIR - SWIR / (NIR + SWIR)
ndwi_median = ndwi_ee_s2(s2_median,geom)

# NDVI
ndvi_median = ndvi_ee_s2(s2_median, geom)
ndvi_focal = ndvi_median.focalMedian(3, 'square', 'pixels')
# NDPI

n = NIR.subtract(RED.multiply(ee.Number(0.74)).add(SWIR.multiply(ee.Number(0.26))))
d = NIR.add(RED.multiply(ee.Number(0.74)).add(SWIR.multiply(ee.Number(0.26))))

ndpi_median =n.divide(d)


# BARE SOIL INDEX (BSI)

n = RED.add(SWIR).subtract(NIR.add(BLUE))
d = RED.add(SWIR).add(NIR.add(BLUE))
bsi_median = n.divide(d)

# SAVI 
L = ee.Number(0.5)
n = NIR.subtract(RED).multiply(ee.Number(1).add(L))
d = NIR.add(RED).add(L)
savi_median = n.divide(d)


# MNDWI
mndwi_median = GREEN.subtract(SWIR).divide(GREEN.add(SWIR))

#IBI
ibi_median = ndbi_median.subtract(
                    savi_median.add(
                        mndwi_median).divide(ee.Number(2))
                    ).divide(
                        ndbi_median.add(
                            savi_median.add(
                             mndwi_median).divide(ee.Number(2))
                        )

                    )


# New Built-up Index
nbi_median = (RED.multiply(SWIR)).divide(NIR)

# Modified Built-up Index (MBI)
mbi_median = SWIR.add(RED).subtract(NIR).divide(SWIR.add(RED).add(NIR).subtract(SWIR2))

# Enhanced Normalized Difference Impervious Surfaces Index (ENDISI)
## ENDISI=(SWIR1+NIR−Blue)/(SWIR1+NIR+Blue)
endisi_median = SWIR.add(NIR).subtract(BLUE).divide(SWIR.add(NIR).add(BLUE))

## Map Spectral Indices and Sentinel-2 Image

In [6]:
evi_minmax = evi_median.reduceRegion(reducer = ee.Reducer.minMax(), geometry= geom, scale =10).getInfo()
evi_minmax


{'EVI_max': 1.2967286982937627, 'EVI_min': -0.7408539549421445}

In [7]:
# Initialize Map
m = geemap.Map()
m.add_basemap('SATELLITE')  # or 'Esri.WorldImagery'
m.centerObject(geom)

# SENTINL-2 RGB
m.addLayer(s2_median, dict(min = 0, max = .3, bands =['B4', 'B3', 'B2']) , 'Sentinel-2 RGB')


# INDICES
# m.addLayer(evi_median, dict(min = evi_minmax['EVI_min'], max = evi_minmax['EVI_max'], palette = ['red','yellow', 'green']) , 'EVI Median')
m.addLayer(ndbi_median, dict(min = -1, max = 1, palette = ['black','yellow', 'red']) , 'NDBI Median')
# m.addLayer(ibi_median, dict(min = -1, max = 1, palette = ['black','yellow', 'red']) , 'IBI Median')
# m.addLayer(ndvi_median, dict(min = -1, max = 1, palette = ['red','yellow', 'green']) , 'NDVI Median')
# m.addLayer(ndvi_focal, dict(min = -1, max = 1, palette = ['red','yellow', 'green']) , 'NDVI Focal')
m.addLayer(endisi_median, dict(min = -1, max = 1, palette = ['black','yellow', 'red']) , 'ENDISI Median')
m.addLayer(mbi_median, dict(min = -1, max = 1, palette = ['black','yellow', 'red']) , 'MBI Median')
m.addLayer(nbi_median, dict(min = -1, max = 1, palette = ['black','yellow', 'red']) , 'NBI Median')

# m.addLayer(ndpi_median, dict(min = -1, max = 1, palette = ['red','yellow', 'green']) , 'NDPI Median')
# m.addLayer(bsi_median, dict(min = -1, max = 1, palette = ['red','yellow', 'green']) , 'BSI Median')
# m.addLayer(savi_median, dict(min = -1, max = 1, palette = ['red','yellow', 'green']) , 'SAVI Median')

# m.addLayer(il_bldg, dict(color = 'red', fill = None), 'Building Footprints')
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

# Download Layers

### Sentinel-2

In [8]:
# geemap.download_ee_image(
#     s2_median,
#     filename=S2_MEDIAN,
#     region =geom,
#     crs=str(utm_crs),
#     scale= 10
# )

### Spectral Indices

In [9]:

# geemap.download_ee_image(
#     ndvi_median,
#     filename=NDVI_MEDIAN,
#     region =geom,
#     crs=str(utm_crs),
#     scale= 10
# )


# geemap.download_ee_image(
#     evi_median,
#     filename=EVI_MEDIAN,
#     region =geom,
#     crs=str(utm_crs),
#     scale= 10,
#     dtype='float32'
# )

# geemap.download_ee_image(
#     bu_median,
#     filename=BU_MEDIAN,
#     region =geom,
#     crs=str(utm_crs),
#     scale= 10
# )


# geemap.download_ee_image(
#     ndwi_median,
#     filename=NDWI_MEDIAN,
#     region =geom,
#     crs=str(utm_crs),
#     scale= 10
# )


# geemap.download_ee_image(
#     ndpi_median,
#     filename=NDPI_MEDIAN,
#     region =geom,
#     crs=str(utm_crs),
#     scale= 10
# )


# geemap.download_ee_image(
#     bsi_median,
#     filename=BSI_MEDIAN,
#     region =geom,
#     crs=str(utm_crs),
#     scale= 10
# )


# geemap.download_ee_image(
#     savi_median,
#     filename=SAVI_MEDIAN,
#     region =geom,
#     crs=str(utm_crs),
#     scale= 10
# )


geemap.download_ee_image(
    mndwi_median,
    filename=MNDWI_MEDIAN,
    region =geom,
    crs=str(utm_crs),
    scale= 10
)

geemap.download_ee_image(
    ibi_median,
    filename=IBI_MEDIAN,
    region =geom,
    crs=str(utm_crs),
    scale= 10
)

geemap.download_ee_image(
    nbi_median,
    filename=NBI_MEDIAN,
    region =geom,
    crs=str(utm_crs),
    scale= 10
)

geemap.download_ee_image(
    mbi_median,
    filename=MBI_MEDIAN,
    region =geom,
    crs=str(utm_crs),
    scale= 10
)


geemap.download_ee_image(
    endisi_median,
    filename=ENDISI_MEDIAN,
    region =geom,
    crs=str(utm_crs),
    scale= 10
)


mndwi_median.tif: |          | 0.00/1.02M (raw) [  0.0%] in 00:00 (eta:     ?)

There is no STAC entry for: None


ibi_median.tif: |          | 0.00/2.04M (raw) [  0.0%] in 00:00 (eta:     ?)

nbi_median.tif: |          | 0.00/1.02M (raw) [  0.0%] in 00:00 (eta:     ?)

mbi_median.tif: |          | 0.00/1.02M (raw) [  0.0%] in 00:00 (eta:     ?)

endisi_median.tif: |          | 0.00/1.02M (raw) [  0.0%] in 00:00 (eta:     ?)

###  Buildings Layer

In [10]:
# geemap.ee_to_geojson(il_bldg, filename=BLDG_FP)